In [3]:
import pandas as pd 

ds_json = pd.read_json('D:/work_dir/Datasets/LoRa_anomaly-detection/data/lorawan_antwerp_2019_dataset.json')

In [4]:
gw_loc_json = pd.read_json('D:/work_dir/Datasets/LoRa_anomaly-detection/data/lorawan_antwerp_gateway_locations.json')

In [5]:
ds_wref = pd.read_csv('D:/work_dir/Datasets/LoRa_anomaly-detection/data/antwerp_combo_raw_ref.csv')

In [10]:
# Checking and extracting values from strings 

import re

value_with_unit = "150 m"
# Use regular expression to extract the numeric part
match = re.match(r"([-+]?\d*\.\d+|[-+]?\d+)", value_with_unit)
if match:
    value_float = float(match.group(0))
    print(value_float)  # Output: 1.0
else:
    print("No numeric value found.")


150.0


In [11]:
# Getting average building height

import requests
import re

def get_average_building_height(lat1, lon1, lat2, lon2):
    # Define the Overpass API query
    query = f"""
    [out:json];
    (
      way["building"]["height"]({lat1},{lon1},{lat2},{lon2});
    );
    out body;
    """
    
    # Send the request to the Overpass API
    response = requests.get("http://overpass-api.de/api/interpreter", params={'data': query})
    data = response.json()
    
    # Extract building heights
    heights = []
    for element in data['elements']:
        if 'tags' in element and 'height' in element['tags']:
            h = element['tags']['height']
            # The height elements are strings and may contain the units like "150 m"
            # So with the help of regular expression check we only take the value
            match = re.match(r"([-+]?\d*\.\d+|[-+]?\d+)", h)
            height = match.group(0)
            heights.append(float(height))
    
    # Calculate average height
    if heights:
        average_height = sum(heights) / len(heights)
        return average_height
    else:
        return None  # No building height data available

# Example coordinates (replace with actual values)
lat1, lon1 = 51.5074, -0.1278  # Location 1 (e.g., London)
lat2, lon2 = 51.5155, -0.0922  # Location 2 (e.g., another point in London)

average_height = get_average_building_height(lat1, lon1, lat2, lon2)
print("Average Building Height:", average_height)


Average Building Height: 44.8875


In [ ]:
import requests
from shapely.geometry import Point
from shapely.ops import nearest_points
import numpy as np

def get_buildings(lat1, lon1, lat2, lon2):
    query = f"""
    [out:json];
    (
      way["building"]({lat1},{lon1},{lat2},{lon2});
    );
    out body;
    """
    
    response = requests.get("http://overpass-api.de/api/interpreter", params={'data': query})
    data = response.json()
    
    # Extract building coordinates
    buildings = []
    for element in data['elements']:
        if 'geometry' in element:
            coords = [(point['lat'], point['lon']) for point in element['geometry']]
            centroid = Point(np.mean([p[0] for p in coords]), np.mean([p[1] for p in coords]))
            buildings.append(centroid)
    
    return buildings

def average_building_separation(buildings):
    if len(buildings) < 2:
        return 0
    
    distances = []
    for i in range(len(buildings)):
        for j in range(i + 1, len(buildings)):
            distance = buildings[i].distance(buildings[j])
            distances.append(distance)
    
    return np.mean(distances)

# Example coordinates (replace with actual values)
lat1, lon1 = 51.5074, -0.1278  # Location 1 (e.g., London)
lat2, lon2 = 51.5155, -0.0922  # Location 2 (e.g., another point in London)

buildings = get_buildings(lat1, lon1, lat2, lon2)
avg_separation = average_building_separation(buildings)
print("Average Building Separation (in degrees):", avg_separation)


In [ ]:
import requests
import numpy as np

def get_average_street_width(lat1, lon1, lat2, lon2):
    # Define the Overpass API query to get streets with width
    query = f"""
    [out:json];
    (
      way["highway"]["width"]({lat1},{lon1},{lat2},{lon2});
    );
    out body;
    """
    
    # Send the request to the Overpass API
    response = requests.get("http://overpass-api.de/api/interpreter", params={'data': query})
    
    # Check if the request was successful
    if response.status_code != 200:
        print("Error fetching data from Overpass API")
        return None
    
    data = response.json()
    
    # Extract street widths
    widths = []
    for element in data['elements']:
        if 'tags' in element and 'width' in element['tags']:
            try:
                widths.append(float(element['tags']['width']))  # Convert width to float
            except ValueError:
                continue  # Skip if conversion fails

    # Calculate average width
    if widths:
        average_width = np.mean(widths)
        return average_width
    else:
        return None  # No width data available

# Example coordinates (replace with actual values)
lat1, lon1 = 51.5074, -0.1278  # Location 1 (e.g., London)
lat2, lon2 = 51.5155, -0.0922  # Location 2 (e.g., another point in London)

average_width = get_average_street_width(lat1, lon1, lat2, lon2)
if average_width is not None:
    print("Average Street Width (in meters):", average_width)
else:
    print("No street width data available in this area.")
